In [ ]:
import numpy as np
import pandas as pd
import tifffile as tf
import matplotlib.pyplot as plt
from pathlib import Path
from map_spots_to_masks import keep_dots_in_cells, keep_dots_parallel

# Map Spots to Cells

In [ ]:
#how many z slices?
num_z=5
#paths for decoded genes and mask
num_pos = np.arange(0,60,1)
if num_z > 1:
    for c in [1,2,3]:
        for z in range(num_z):
            decoded_genes = []
            mask_paths = []
            for i in num_pos:
                decoded_genes.append(f"/groups/CaiLab/personal/Lex/raw/102322_10K_NIH3T3/seqFISH_datapipeline/output/dots_detected/Channel_{c}/Pos{i}/locations_z_{z}.csv")
                mask_paths.append(f'/groups/CaiLab/personal/Lex/raw/102322_10K_NIH3T3/seqFISH_datapipeline/output/edges_deleted/MMStack_Pos{i}_z{z}.tif')
            keep_dots_parallel(mask_paths,decoded_genes)
else:
    decoded_genes = []
    mask_paths = []
    for c in [1,2,3]:
        for i in num_pos:
            decoded_genes.append(f"/groups/CaiLab/personal/Lex/raw/092222_150_nih_3t3/seqFISH_datapipeline/output/dots_detected/Channel_{c}/Pos{i}/locations_z_0.csv")
            mask_paths.append(f'/groups/CaiLab/personal/Lex/raw/092222_150_nih_3t3/seqFISH_datapipeline/output/edges_deleted/MMStack_Pos{i}.tif')
        keep_dots_parallel(mask_paths,decoded_genes)

# If encoding across channels use this for combining spots across channels

In [ ]:
from tqdm import tqdm

#name of experiment directory
exp_dir = "102322_10K_NIH3T3"
#define channels used
channel = [1,2,3]
#define total number of pos
pos_tot = 60
#define number of z's
num_z = 5
#for each channel in a specific position, grab the locations file
#then, concatenate those files and write it out
for i in tqdm(range(pos_tot)):
    for z in range(num_z):
        df_list = []
        for c in channel:
            paths = f"/groups/CaiLab/personal/Lex/raw/{exp_dir}/seqFISH_datapipeline/output/dots_detected/Channel_{c}/spots_in_cells/Pos{i}/locations_z_{z}.csv"
            try:
                df = pd.read_csv(paths)
            except:
                continue
            df_list.append(df)
        try:
            df_comb = pd.concat(df_list).reset_index(drop=True)
        except:
            continue
        output= Path(f"/groups/CaiLab/personal/Lex/raw/{exp_dir}/seqFISH_datapipeline/output/dots_detected/Channel_All/Pos{i}")
        output.mkdir(parents=True,exist_ok=True)
        output = output / f"locations_z_{z}.csv"
        df_comb.to_csv(str(output))